In [29]:
#%reload_ext autoreload
#%autoreload 2
import logging
log = lambda msg: logging.info(msg)
import os
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Building Day Pipeline

In [30]:
input_file = "C:\\dev\\play\\brainwave-data-day\\2024-11-15-09-26-33\\raw.fif"

In [31]:
import pandas as pd

input_dir = "C:\\dev\\play\\brainwave-data-day"
stats_df = pd.read_csv(input_dir + os.path.sep + "day_stats.csv")

In [32]:
from models.eeg_states.eeg_states import load_and_prepare_day_data_energy_eeg_state_events

events = load_and_prepare_day_data_energy_eeg_state_events()

C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:115: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-11-15 09:34:49.320000+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  events.at[i, 'since'] = since_time
C:\dev\play\brainwave-processor\models\eeg_states\eeg_states.py:117: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  events['duration'] = events['timestamp'] - events['since']


In [33]:
import io
import contextlib
from datetime import timedelta
import convert 

# Load MNE
output_buffer = io.StringIO()
with contextlib.redirect_stdout(output_buffer), contextlib.redirect_stderr(output_buffer):
    log("Loading MNE file " + input_file)
    raw, input_file_without_ext, mne_filtered = convert.load_mne_file(log, input_file)
    
    channels = raw.info['ch_names']
    sfreq = raw.info['sfreq']
    start_date = raw.info['meas_date']
    duration = timedelta(seconds=float(raw.times[-1]))
    end_date = start_date + duration
    
    log(f"Start date: {start_date} end {end_date} duration {duration} channels: {channels} sfreq: {sfreq}")


2024-11-16 13:43:48,631 - INFO - Loading MNE file C:\dev\play\brainwave-data-day\2024-11-15-09-26-33\raw.fif
2024-11-16 13:43:48,632 - INFO - Reading file C:\dev\play\brainwave-data-day\2024-11-15-09-26-33\raw.fif
2024-11-16 13:43:48,859 - INFO - Finished reading file C:\dev\play\brainwave-data-day\2024-11-15-09-26-33\raw.fif
2024-11-16 13:43:49,055 - INFO - Start date: 2024-11-15 09:26:33.913368+00:00 end 2024-11-15 11:48:58.029368+00:00 duration 2:22:24.116000 channels: ['Fpz-M1'] sfreq: 250.0


In [34]:
from memory import garbage_collect

# Save as EDF
garbage_collect(log)
log("Saving as EDF")
convert.save_mne_as_downsample_edf(log, mne_filtered, input_file_without_ext)


2024-11-16 13:43:49,200 - INFO - Memory Usage: 534.29 MB GC to 534.29 MB
2024-11-16 13:43:49,201 - INFO - Saving as EDF


Overwriting existing file.


C:\dev\play\brainwave-processor\convert.py:83: RuntimeWarning: EDF format requires equal-length data blocks, so 0.88 seconds of edge values were appended to all channels when writing the final block.
  mne.export.export_raw(input_file_without_ext + ".edf", resampled, overwrite=True)


In [35]:
import yasa_features

# YASA features
garbage_collect(log)
log("Extracting YASA features")
yasa_feats, channel_feats_dict = yasa_features.extract_yasa_features2(log, channels, mne_filtered)


2024-11-16 13:43:50,349 - INFO - Memory Usage: 670.55 MB GC to 670.55 MB
2024-11-16 13:43:50,350 - INFO - Extracting YASA features


In [36]:
# Assuming the index represents the epoch
df = yasa_feats
df['EpochTime'] = (df.index * 30) + start_date.timestamp()
df['TimestampUK'] = pd.to_datetime(df['EpochTime'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Europe/London')
df.drop('EpochTime', axis=1, inplace=True)

In [37]:
df

,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,Fpz-M1_eeg_alphaab,...,Fpz-M1_eeg_thetaabs,Fpz-M1_eeg_thetaabs_c7min_norm,Fpz-M1_eeg_thetaabs_p2min_norm,Fpz-M1_eeg_thetaabsaa,Fpz-M1_eeg_thetaabsaa_c7min_norm,Fpz-M1_eeg_thetaabsaa_p2min_norm,Fpz-M1_eeg_thetaabsab,Fpz-M1_eeg_thetaabsab_c7min_norm,Fpz-M1_eeg_thetaabsab_p2min_norm,TimestampUK
epoch,,,,,,,,,,,,,,,,,,,,,
0,6.176673e-09,0.440779,0.938026,0.010253,-0.187265,-0.317793,0.015990,-0.107151,-0.214304,1.012075,...,3.604625e-10,0.169903,0.340567,4.594962e-10,0.177583,0.405584,6.204930e-09,0.454050,0.962635,2024-11-15 09:26:33.913367987+00:00
1,4.793729e-09,0.354770,0.766664,0.011490,-0.160262,-0.283900,0.016814,-0.087451,-0.204782,1.046799,...,4.846242e-10,0.129013,0.520255,5.616700e-10,0.136143,0.527961,4.744175e-09,0.364264,0.780841,2024-11-15 09:27:03.913367987+00:00
2,2.732233e-09,0.294199,0.539248,0.012872,-0.141557,-0.247349,0.022083,-0.072530,-0.161011,0.975427,...,2.179857e-10,0.084185,0.322897,2.777870e-10,0.093102,0.342075,2.606673e-09,0.301919,0.542899,2024-11-15 09:27:33.913367987+00:00
3,4.477972e-09,0.262025,0.533698,0.010234,-0.122803,-0.265209,0.015935,-0.060902,-0.174652,0.985904,...,3.633763e-10,0.069282,0.329423,4.357072e-10,0.076593,0.343706,4.428800e-09,0.271327,0.537311,2024-11-15 09:28:03.913367987+00:00
4,5.564690e-10,0.230852,0.185495,0.025359,-0.097883,-0.058277,0.049332,-0.046370,0.018017,0.967655,...,6.372231e-11,0.066686,0.114700,9.072085e-11,0.070905,0.122857,5.157063e-10,0.241380,0.183293,2024-11-15 09:28:33.913367987+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,5.698911e-10,-0.659453,-0.479267,0.022562,0.270177,0.413500,0.043066,0.396677,0.493923,1.011983,...,4.257816e-11,-0.724951,-0.583135,6.656839e-11,-0.650659,-0.546869,5.511613e-10,-0.666912,-0.473799,2024-11-15 11:46:03.913367987+00:00
280,4.251687e-10,-0.681057,-0.468223,0.014979,0.303907,0.246545,0.037027,0.441196,0.384507,0.965190,...,2.641711e-11,-0.745837,-0.578704,4.229134e-11,-0.668681,-0.542340,4.073702e-10,-0.689108,-0.462835,2024-11-15 11:46:33.913367987+00:00
281,4.524738e-10,-0.703470,-0.472234,0.021224,0.361342,0.172114,0.039850,0.497981,0.359070,0.958690,...,3.367944e-11,-0.763744,-0.588910,5.185284e-11,-0.683726,-0.553377,4.275219e-10,-0.712160,-0.466583,2024-11-15 11:47:03.913367987+00:00


In [38]:
import scaling

# Scaled
scale_by_stats = scaling.scale_by_stats(df, stats_df)
df = df.join(scale_by_stats.add_suffix('_s'))


In [39]:
df

,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,Fpz-M1_eeg_alphaab,...,Fpz-M1_eeg_thetaab_p2min_norm_s,Fpz-M1_eeg_thetaabs_s,Fpz-M1_eeg_thetaabs_c7min_norm_s,Fpz-M1_eeg_thetaabs_p2min_norm_s,Fpz-M1_eeg_thetaabsaa_s,Fpz-M1_eeg_thetaabsaa_c7min_norm_s,Fpz-M1_eeg_thetaabsaa_p2min_norm_s,Fpz-M1_eeg_thetaabsab_s,Fpz-M1_eeg_thetaabsab_c7min_norm_s,Fpz-M1_eeg_thetaabsab_p2min_norm_s
epoch,,,,,,,,,,,,,,,,,,,,,
0,6.176673e-09,0.440779,0.938026,0.010253,-0.187265,-0.317793,0.015990,-0.107151,-0.214304,1.012075,...,0.943667,0.748281,0.857868,0.883320,0.819610,0.907524,0.961685,1.388875,1.333190,1.730974
1,4.793729e-09,0.354770,0.766664,0.011490,-0.160262,-0.283900,0.016814,-0.087451,-0.204782,1.046799,...,1.169581,1.056810,0.795293,1.110780,1.042343,0.836390,1.121932,1.032270,1.184764,1.479508
2,2.732233e-09,0.294199,0.539248,0.012872,-0.141557,-0.247349,0.022083,-0.072530,-0.161011,0.975427,...,0.917664,0.394242,0.726692,0.860951,0.423494,0.762508,0.878525,0.510455,1.081700,1.150375
3,4.477972e-09,0.262025,0.533698,0.010234,-0.122803,-0.265209,0.015935,-0.060902,-0.174652,0.985904,...,0.835942,0.755522,0.703885,0.869213,0.767751,0.734170,0.880660,0.955279,1.031129,1.142646
4,5.564690e-10,0.230852,0.185495,0.025359,-0.097883,-0.058277,0.049332,-0.046370,0.018017,0.967655,...,0.635144,0.010914,0.699913,0.597404,0.015701,0.724405,0.591472,0.000000,0.981623,0.652952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,5.698911e-10,-0.659453,-0.479267,0.022562,0.270177,0.413500,0.043066,0.396677,0.493923,1.011983,...,0.206787,-0.041627,-0.511541,-0.285960,-0.036950,-0.514189,-0.285493,0.008655,-0.519887,-0.255969
280,4.251687e-10,-0.681057,-0.468223,0.014979,0.303907,0.246545,0.037027,0.441196,0.384507,0.965190,...,0.388709,-0.081786,-0.543502,-0.280352,-0.089872,-0.545126,-0.279562,-0.026447,-0.556580,-0.240803
281,4.524738e-10,-0.703470,-0.472234,0.021224,0.361342,0.172114,0.039850,0.497981,0.359070,0.958690,...,0.315134,-0.063740,-0.570906,-0.293270,-0.069029,-0.570950,-0.294015,-0.021528,-0.594688,-0.245987


In [40]:

# Main channel
df['Source'] = channels[0]
df = df.copy()
scaling.add_main_channel(df)


C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

In [41]:
df

,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,Fpz-M1_eeg_alphaab,...,Main_eeg_thetaab_p2min_norm_s,Main_eeg_thetaabs_s,Main_eeg_thetaabs_c7min_norm_s,Main_eeg_thetaabs_p2min_norm_s,Main_eeg_thetaabsaa_s,Main_eeg_thetaabsaa_c7min_norm_s,Main_eeg_thetaabsaa_p2min_norm_s,Main_eeg_thetaabsab_s,Main_eeg_thetaabsab_c7min_norm_s,Main_eeg_thetaabsab_p2min_norm_s
epoch,,,,,,,,,,,,,,,,,,,,,
0,6.176673e-09,0.440779,0.938026,0.010253,-0.187265,-0.317793,0.015990,-0.107151,-0.214304,1.012075,...,0.943667,0.748281,0.857868,0.883320,0.819610,0.907524,0.961685,1.388875,1.333190,1.730974
1,4.793729e-09,0.354770,0.766664,0.011490,-0.160262,-0.283900,0.016814,-0.087451,-0.204782,1.046799,...,1.169581,1.056810,0.795293,1.110780,1.042343,0.836390,1.121932,1.032270,1.184764,1.479508
2,2.732233e-09,0.294199,0.539248,0.012872,-0.141557,-0.247349,0.022083,-0.072530,-0.161011,0.975427,...,0.917664,0.394242,0.726692,0.860951,0.423494,0.762508,0.878525,0.510455,1.081700,1.150375
3,4.477972e-09,0.262025,0.533698,0.010234,-0.122803,-0.265209,0.015935,-0.060902,-0.174652,0.985904,...,0.835942,0.755522,0.703885,0.869213,0.767751,0.734170,0.880660,0.955279,1.031129,1.142646
4,5.564690e-10,0.230852,0.185495,0.025359,-0.097883,-0.058277,0.049332,-0.046370,0.018017,0.967655,...,0.635144,0.010914,0.699913,0.597404,0.015701,0.724405,0.591472,0.000000,0.981623,0.652952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,5.698911e-10,-0.659453,-0.479267,0.022562,0.270177,0.413500,0.043066,0.396677,0.493923,1.011983,...,0.206787,-0.041627,-0.511541,-0.285960,-0.036950,-0.514189,-0.285493,0.008655,-0.519887,-0.255969
280,4.251687e-10,-0.681057,-0.468223,0.014979,0.303907,0.246545,0.037027,0.441196,0.384507,0.965190,...,0.388709,-0.081786,-0.543502,-0.280352,-0.089872,-0.545126,-0.279562,-0.026447,-0.556580,-0.240803
281,4.524738e-10,-0.703470,-0.472234,0.021224,0.361342,0.172114,0.039850,0.497981,0.359070,0.958690,...,0.315134,-0.063740,-0.570906,-0.293270,-0.069029,-0.570950,-0.294015,-0.021528,-0.594688,-0.245987


In [42]:
columns = df.columns.tolist()
columns.insert(0, columns.pop(columns.index('TimestampUK')))
df = df.reindex(columns=columns)

In [43]:
columns

['TimestampUK',
 'Fpz-M1_eeg_abspow',
 'Fpz-M1_eeg_abspow_c7min_norm',
 'Fpz-M1_eeg_abspow_p2min_norm',
 'Fpz-M1_eeg_alpha',
 'Fpz-M1_eeg_alpha_c7min_norm',
 'Fpz-M1_eeg_alpha_p2min_norm',
 'Fpz-M1_eeg_alphaaa',
 'Fpz-M1_eeg_alphaaa_c7min_norm',
 'Fpz-M1_eeg_alphaaa_p2min_norm',
 'Fpz-M1_eeg_alphaab',
 'Fpz-M1_eeg_alphaab_c7min_norm',
 'Fpz-M1_eeg_alphaab_p2min_norm',
 'Fpz-M1_eeg_alphaabs',
 'Fpz-M1_eeg_alphaabs_c7min_norm',
 'Fpz-M1_eeg_alphaabs_p2min_norm',
 'Fpz-M1_eeg_alphaabsaa',
 'Fpz-M1_eeg_alphaabsaa_c7min_norm',
 'Fpz-M1_eeg_alphaabsaa_p2min_norm',
 'Fpz-M1_eeg_alphaabsab',
 'Fpz-M1_eeg_alphaabsab_c7min_norm',
 'Fpz-M1_eeg_alphaabsab_p2min_norm',
 'Fpz-M1_eeg_at',
 'Fpz-M1_eeg_at_c7min_norm',
 'Fpz-M1_eeg_at_p2min_norm',
 'Fpz-M1_eeg_auc',
 'Fpz-M1_eeg_auc_c7min_norm',
 'Fpz-M1_eeg_auc_p2min_norm',
 'Fpz-M1_eeg_beta',
 'Fpz-M1_eeg_beta_c7min_norm',
 'Fpz-M1_eeg_beta_p2min_norm',
 'Fpz-M1_eeg_betaabs',
 'Fpz-M1_eeg_betaabs_c7min_norm',
 'Fpz-M1_eeg_betaabs_p2min_norm',
 'Fpz-M

In [44]:
df

,TimestampUK,Fpz-M1_eeg_abspow,Fpz-M1_eeg_abspow_c7min_norm,Fpz-M1_eeg_abspow_p2min_norm,Fpz-M1_eeg_alpha,Fpz-M1_eeg_alpha_c7min_norm,Fpz-M1_eeg_alpha_p2min_norm,Fpz-M1_eeg_alphaaa,Fpz-M1_eeg_alphaaa_c7min_norm,Fpz-M1_eeg_alphaaa_p2min_norm,...,Main_eeg_thetaab_p2min_norm_s,Main_eeg_thetaabs_s,Main_eeg_thetaabs_c7min_norm_s,Main_eeg_thetaabs_p2min_norm_s,Main_eeg_thetaabsaa_s,Main_eeg_thetaabsaa_c7min_norm_s,Main_eeg_thetaabsaa_p2min_norm_s,Main_eeg_thetaabsab_s,Main_eeg_thetaabsab_c7min_norm_s,Main_eeg_thetaabsab_p2min_norm_s
epoch,,,,,,,,,,,,,,,,,,,,,
0,2024-11-15 09:26:33.913367987+00:00,6.176673e-09,0.440779,0.938026,0.010253,-0.187265,-0.317793,0.015990,-0.107151,-0.214304,...,0.943667,0.748281,0.857868,0.883320,0.819610,0.907524,0.961685,1.388875,1.333190,1.730974
1,2024-11-15 09:27:03.913367987+00:00,4.793729e-09,0.354770,0.766664,0.011490,-0.160262,-0.283900,0.016814,-0.087451,-0.204782,...,1.169581,1.056810,0.795293,1.110780,1.042343,0.836390,1.121932,1.032270,1.184764,1.479508
2,2024-11-15 09:27:33.913367987+00:00,2.732233e-09,0.294199,0.539248,0.012872,-0.141557,-0.247349,0.022083,-0.072530,-0.161011,...,0.917664,0.394242,0.726692,0.860951,0.423494,0.762508,0.878525,0.510455,1.081700,1.150375
3,2024-11-15 09:28:03.913367987+00:00,4.477972e-09,0.262025,0.533698,0.010234,-0.122803,-0.265209,0.015935,-0.060902,-0.174652,...,0.835942,0.755522,0.703885,0.869213,0.767751,0.734170,0.880660,0.955279,1.031129,1.142646
4,2024-11-15 09:28:33.913367987+00:00,5.564690e-10,0.230852,0.185495,0.025359,-0.097883,-0.058277,0.049332,-0.046370,0.018017,...,0.635144,0.010914,0.699913,0.597404,0.015701,0.724405,0.591472,0.000000,0.981623,0.652952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,2024-11-15 11:46:03.913367987+00:00,5.698911e-10,-0.659453,-0.479267,0.022562,0.270177,0.413500,0.043066,0.396677,0.493923,...,0.206787,-0.041627,-0.511541,-0.285960,-0.036950,-0.514189,-0.285493,0.008655,-0.519887,-0.255969
280,2024-11-15 11:46:33.913367987+00:00,4.251687e-10,-0.681057,-0.468223,0.014979,0.303907,0.246545,0.037027,0.441196,0.384507,...,0.388709,-0.081786,-0.543502,-0.280352,-0.089872,-0.545126,-0.279562,-0.026447,-0.556580,-0.240803
281,2024-11-15 11:47:03.913367987+00:00,4.524738e-10,-0.703470,-0.472234,0.021224,0.361342,0.172114,0.039850,0.497981,0.359070,...,0.315134,-0.063740,-0.570906,-0.293270,-0.069029,-0.570950,-0.294015,-0.021528,-0.594688,-0.245987


In [45]:
df['TimestampUK']

epoch
0     2024-11-15 09:26:33.913367987+00:00
1     2024-11-15 09:27:03.913367987+00:00
2     2024-11-15 09:27:33.913367987+00:00
3     2024-11-15 09:28:03.913367987+00:00
4     2024-11-15 09:28:33.913367987+00:00
                      ...                
279   2024-11-15 11:46:03.913367987+00:00
280   2024-11-15 11:46:33.913367987+00:00
281   2024-11-15 11:47:03.913367987+00:00
282   2024-11-15 11:47:33.913367987+00:00
283   2024-11-15 11:48:03.913367987+00:00
Name: TimestampUK, Length: 284, dtype: datetime64[ns, Europe/London]

In [46]:
df
# Remvoe when YASA-esque model restored
output_csv_file = input_file_without_ext + ".output.csv"
df.to_csv(output_csv_file)

log("All done! " + input_file)


2024-11-16 13:43:53,195 - INFO - All done! C:\dev\play\brainwave-data-day\2024-11-15-09-26-33\raw.fif


In [47]:
from catboost import CatBoostClassifier
from models.eeg_states.eeg_states_model import predict_only_day_energy_model_pipeline

# DayEnergy model
models_and_data = [predict_only_day_energy_model_pipeline('ignored', df, False)]
model = CatBoostClassifier()
model.load_model("models/day_data/day-energy-non-realtime_catboost_model.cbm")
predictions = model.predict_proba(models_and_data[0].X)
predictions_df = pd.DataFrame(predictions, index=models_and_data[0].X.index)
predictions_df['DayEnergyPrediction'] = predictions_df[1]
predictions_df.drop([0, 1], axis=1, inplace=True)
df = pd.concat([df, predictions_df], axis=1)


['Main_eeg_abspow_s', 'Main_eeg_abspow_c7min_norm_s', 'Main_eeg_abspow_p2min_norm_s', 'Main_eeg_alpha_s', 'Main_eeg_alpha_c7min_norm_s', 'Main_eeg_alpha_p2min_norm_s', 'Main_eeg_alphaaa_s', 'Main_eeg_alphaaa_c7min_norm_s', 'Main_eeg_alphaaa_p2min_norm_s', 'Main_eeg_alphaab_s', 'Main_eeg_alphaab_c7min_norm_s', 'Main_eeg_alphaab_p2min_norm_s', 'Main_eeg_alphaabs_s', 'Main_eeg_alphaabs_c7min_norm_s', 'Main_eeg_alphaabs_p2min_norm_s', 'Main_eeg_alphaabsaa_s', 'Main_eeg_alphaabsaa_c7min_norm_s', 'Main_eeg_alphaabsaa_p2min_norm_s', 'Main_eeg_alphaabsab_s', 'Main_eeg_alphaabsab_c7min_norm_s', 'Main_eeg_alphaabsab_p2min_norm_s', 'Main_eeg_at_s', 'Main_eeg_at_c7min_norm_s', 'Main_eeg_at_p2min_norm_s', 'Main_eeg_auc_s', 'Main_eeg_auc_c7min_norm_s', 'Main_eeg_auc_p2min_norm_s', 'Main_eeg_beta_s', 'Main_eeg_beta_c7min_norm_s', 'Main_eeg_beta_p2min_norm_s', 'Main_eeg_betaabs_s', 'Main_eeg_betaabs_c7min_norm_s', 'Main_eeg_betaabs_p2min_norm_s', 'Main_eeg_db_s', 'Main_eeg_db_c7min_norm_s', 'Main_eeg_

In [57]:
from models.eeg_states.eeg_states import process_row
import pandas as pd
from tqdm import tqdm
df['DayEnergyManual'] = None

# from models.eeg_states.eeg_states_model import day_energy_mapping
day_energy_mapping = {
    'near worst': 0.0,
    'struggling': 0.1,
    'tired': 0.3,
    'standard tired': 0.4,
    'okish': 0.7,
    'lockable': 0.9,
    'great': 1.0
}

# Iterate over each row in yasa_df
for i, yasa_row in tqdm(df.iterrows(), total=df.shape[0]):
    epoch_type, matched_night_event = process_row(yasa_row, events)
    if epoch_type is not None:
        df.at[i, 'DayEnergyManual'] = day_energy_mapping[epoch_type]


100%|██████████| 284/284 [00:00<00:00, 878.22it/s] 


In [49]:
events

,version,phase,event,duration,TimestampUK,SinceUK,dayAndNightOf
0,v1,day,tired_long,0 days 00:05:00,2024-11-15 09:39:49.320000+00:00,2024-11-15 09:34:49.320000+00:00,2024-11-15
1,v1,day,tired_long,0 days 00:02:50.530000,2024-11-15 09:42:39.850000+00:00,2024-11-15 09:39:49.320000+00:00,2024-11-15
2,v1,day,tired_long,0 days 00:01:14.072000,2024-11-15 09:43:53.922000+00:00,2024-11-15 09:42:39.850000+00:00,2024-11-15
3,v1,day,tired_long,0 days 00:03:34.874000,2024-11-15 09:47:28.796000+00:00,2024-11-15 09:43:53.922000+00:00,2024-11-15
4,v1,day,tired_long,0 days 00:03:59.597000,2024-11-15 09:51:28.393000+00:00,2024-11-15 09:47:28.796000+00:00,2024-11-15
...,...,...,...,...,...,...,...
60,v1,day,standard tired,NaT,2024-11-16 13:30:35.783000+00:00,NaT,2024-11-16
61,v1,day,lockable,NaT,2024-11-16 13:34:00.897000+00:00,NaT,2024-11-16
62,v1,day,great,NaT,2024-11-16 13:38:47.890000+00:00,NaT,2024-11-16
63,v1,day,great_long,0 days 00:01:05.356000,2024-11-16 13:39:53.246000+00:00,2024-11-16 13:38:47.890000+00:00,2024-11-16


In [58]:
df[['TimestampUK', 'DayEnergyPrediction', 'DayEnergyManual']]

,TimestampUK,DayEnergyPrediction,DayEnergyManual
epoch,,,
0,2024-11-15 09:26:33.913367987+00:00,0.571640,None
1,2024-11-15 09:27:03.913367987+00:00,0.571624,None
2,2024-11-15 09:27:33.913367987+00:00,0.571790,None
3,2024-11-15 09:28:03.913367987+00:00,0.572564,None
4,2024-11-15 09:28:33.913367987+00:00,0.572135,None
...,...,...,...
279,2024-11-15 11:46:03.913367987+00:00,0.524848,None
280,2024-11-15 11:46:33.913367987+00:00,0.522384,None
281,2024-11-15 11:47:03.913367987+00:00,0.520068,0.0
